In [ ]:
#@title # **下載**
#@markdown 選擇一種下載模式並貼上鏈接。

import os
import re
import requests
import subprocess

# --- 初始化環境 (只在第一次運行時執行) ---
if not os.path.exists("/content/DL"):
    print("正在初始化環境...")
    !mkdir -p /content/DL
    !rm -rf /content/sample_data
    !apt-get install -y aria2 > /dev/null 2>&1
    !pip install uv > /dev/null 2>&1
    !uv pip install gigafile --system > /dev/null 2>&1
    print("環境初始化完成。")

# --- 設定參數 ---
link = "" #@param{type:"string"}
mode = "Torrent" #@param ["Direct Link", "Torrent", "Gigafile", "Pixeldrain", "Gofile"]
mount_google_drive = False #@param {type:"boolean"}

# --- 掛載 Google Drive (選用) ---
if mount_google_drive:
    from google.colab import drive
    if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')
    download_dir = "/content/drive/MyDrive/ColabDL" # 可以修改為你想要的雲端路徑
    if not os.path.exists(download_dir):
        !mkdir -p "{download_dir}"
else:
    download_dir = "/content/DL"

%cd "{download_dir}"
print(f"當前下載目錄: {download_dir}")

# --- 功能函數 ---

def get_trackers():
    """獲取最新的 Tracker 列表以加速 BT 下載"""
    try:
        response = requests.get("https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best.txt")
        return response.text.replace('\n\n', ',').replace('\n', ',')
    except:
        return ""

# --- 下載邏輯 ---

if mode == "Torrent":
    print("正在啟動 Aria2 下載 Torrent...")
    trackers = get_trackers()
    cmd = f'aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true --bt-tracker="{trackers}" "{link}"'
    !{cmd}

elif mode == "Direct Link":
    print("正在啟動 Aria2 下載直鏈...")
    # 增加 --check-certificate=false 以防某些過期證書網站報錯
    !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 --check-certificate=false "{link}"

elif mode == "Gigafile":
    print("正在使用 Gigafile 下載...")
    !gfile download "{link}"

elif mode == "Pixeldrain":
    print("正在處理 Pixeldrain 鏈接...")
    # 更安全的 ID 提取方式
    if '/u/' in link:
        pd_id = link.split('/u/')[-1].split('/')[0] # 處理可能存在的尾部斜線
        bypass = f'https://pixeldrain.com/api/file/{pd_id}'
        print(f"解析 ID 為: {pd_id}")
        !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{bypass}"
    else:
        print("❌ URL格式錯誤！正確格式應包含 `pixeldrain.com/u/ID`")

elif mode == "Gofile":
    print("正在處理 Gofile...")
    gofile_tool_path = "/content/gofile-downloader"
    if not os.path.exists(gofile_tool_path):
        print("安裝 Gofile 下載器中...")
        !git clone https://github.com/ltsdw/gofile-downloader.git "{gofile_tool_path}"
        !uv pip install -r "{gofile_tool_path}/requirements.txt" --system

    # 使用絕對路徑運行，不切換目錄
    !python "{gofile_tool_path}/gofile-downloader.py" "{link}"

print("\n✅ 任務完成 Done!")

---
# **上傳**

In [ ]:
#@title # **上傳到Buzzheavier**
import json
import urllib.parse
import os

filename = ""  #@param {type:"string"}

# 1. 檢查檔案是否存在
if not filename or not os.path.isfile(filename):
    raise FileNotFoundError(f"檔案不存在或未指定：{filename!r}")

# 2. URL 只取檔名（避免完整路徑被帶入網址）
basename = os.path.basename(filename)
url = f"https://w.buzzheavier.com/{urllib.parse.quote(basename)}"

# 3. 上傳
result = !curl -#o - -T "{filename}" "{url}" | cat

# 4. 解析回應
try:
    response = json.loads(result[-1])
except (json.JSONDecodeError, IndexError):
    raise RuntimeError(f"無法解析伺服器回應：{result}")

if response.get("code") == 201:
    data = response["data"]
    print(f"✅ 上傳成功！")
    print(f"   檔名：{data['name']}")
    print(f"   大小：{data['size']:,} bytes")
    print(f"   到期：{data.get('expiry', 'N/A')}")
    print(f"   下載網址：https://buzzheavier.com/{data['id']}")
else:
    raise RuntimeError(f"上傳失敗 (code={response.get('code')})：{response}")

In [ ]:
#@title # **上傳到Pixeldrain**
#@markdown **Pixeldrain上限 [10](https://pixeldrain.com/) GB**
import os
import requests
from tqdm.auto import tqdm
from IPython.display import Image, display

#@markdown 現在需要帳號ㄌ！
filename = "" #@param{type:"string"}
pd_api_key = "" #@param{type:"string"}
bypass_pixeldrain = False #@param{type:"boolean"}

# ─── 串流上傳包裝器（每次只讀取一小塊，不會載入整個檔案） ───
class StreamUpload:
    def __init__(self, filepath):
        self.file_size = os.path.getsize(filepath)
        self._file = open(filepath, 'rb')
        self.pbar = tqdm(
            total=self.file_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
            desc='📤 上傳中',
            bar_format='{desc} {percentage:3.1f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        )

    def read(self, size=-1):
        chunk = self._file.read(size)
        self.pbar.update(len(chunk))
        return chunk

    def __len__(self):
        return self.file_size        # 讓 requests 自動設定 Content-Length

    def close(self):
        self._file.close()
        self.pbar.close()

# ─── 主程式 ───
if pd_api_key == "":
    print("❌ 請填入 API 密鑰！")
    print("👉 取得密鑰：https://pixeldrain.com/user/api_keys")

elif not os.path.exists(filename):
    print(f"❌ 找不到檔案「{filename}」，請確認路徑與檔名。")

else:
    file_size = os.path.getsize(filename)
    file_size_gb = file_size / (1024 ** 3)

    # 超過 10GB 預先警告
    if file_size_gb > 10:
        print(f"⚠️ 檔案大小 {file_size_gb:.2f} GB 超過 Pixeldrain 的 10 GB 限制，上傳可能會失敗。")

    file_size_mb = file_size / (1024 ** 2)
    print(f"📁 檔案：{filename}（{file_size_mb:.1f} MB）\n")

    stream = StreamUpload(filename)
    try:
        response = requests.put(
            f"https://pixeldrain.com/api/file/{os.path.basename(filename)}",
            auth=("", pd_api_key),
            data=stream,                                      # ← 串流傳送，不會一次載入記憶體
            headers={'Content-Type': 'application/octet-stream'}
        )

        if response.status_code in (200, 201):
            res = response.json()
            file_id = res.get('id')
            if file_id:
                print(f"\n✅ 上傳成功！")
                print(f"🔗 下載網址：https://pixeldrain.com/u/{file_id}")
            else:
                print(f"\n⚠️ 回應異常：{res}")
        else:
            print(f"\n❌ 上傳失敗（HTTP {response.status_code}）")
            try:
                err_msg = response.json().get('message', response.text)
            except Exception:
                err_msg = response.text
            print(f"   錯誤訊息：{err_msg}")

    except requests.exceptions.ConnectionError:
        print("\n❌ 網路連線失敗，請檢查網路狀態。")
    except Exception as e:
        print(f"\n❌ 發生錯誤：{e}")
    finally:
        stream.close()                                        # ← 確保無論成功或失敗都會釋放資源

# ─── Bypass 提醒 ───
if bypass_pixeldrain:
    print("\n⚠️ 提醒：目前無法使用 Bypass 功能")
    display(Image(url='https://i.imgur.com/beqdYSg.png', width=500))

In [ ]:
#@title # **上傳到Gofile**
# 先安裝必要的套件
!pip install uv -q
!uv pip install requests-toolbelt tqdm -q

import requests
import json
import os
from requests_toolbelt import MultipartEncoder, MultipartEncoderMonitor
from tqdm import tqdm

filename = "" #@param{type:"string"}
dev_mode = False #@param{type:"boolean"}
# 勾選此項可以清空紀錄，上傳到全新的資料夾
reset_folder = False #@param{type:"boolean"}

# 建立全域變數來記住 Token 和 Folder ID，實現多次執行上傳至同資料夾
if 'gofile_token' not in globals() or reset_folder:
    gofile_token = None
if 'gofile_folder_id' not in globals() or reset_folder:
    gofile_folder_id = None

def upload_huge_file(file_path):
    global gofile_token, gofile_folder_id

    if not os.path.exists(file_path):
        print(f"❌ 錯誤：找不到檔案 '{file_path}'")
        return

    # 1. 取得最佳伺服器
    try:
        server_req = requests.get('https://api.gofile.io/servers')
        server_data = server_req.json()
        best_server = server_data['data']['servers'][0]['name']
        upload_url = f"https://{best_server}.gofile.io/contents/uploadfile"
        print(f"🚀 準備上傳至伺服器: {best_server}")
    except Exception as e:
        print("❌ 無法獲取 Gofile 伺服器列表")
        return

    # 2. 設定串流上傳
    try:
        # 基本欄位：檔案
        fields = {
            'file': (os.path.basename(file_path), open(file_path, 'rb'))
        }

        # 如果有之前的紀錄，把 token 和 folderId 加進去，這樣就會傳到同一個資料夾
        if gofile_token and gofile_folder_id:
            fields['token'] = gofile_token
            fields['folderId'] = gofile_folder_id
            print("🔗 偵測到先前的上傳紀錄，將合併至同一個 Gofile 資料夾中...")

        encoder = MultipartEncoder(fields=fields)

        # 設定進度條
        pbar = tqdm(total=encoder.len, unit='B', unit_scale=True, desc="上傳中")

        def callback(monitor):
            # 更新進度條：目前的位元組數 - 上次更新的位元組數
            pbar.update(monitor.bytes_read - pbar.n)

        monitor = MultipartEncoderMonitor(encoder, callback)

        # 發送請求 (header 必須包含 Content-Type)
        headers = {'Content-Type': monitor.content_type}

        # 發送請求
        response = requests.post(upload_url, data=monitor, headers=headers)
        pbar.close()

        # 3. 處理回應
        response_json = response.json()
        if response_json['status'] == 'ok':
            data = response_json['data']

            # 儲存 Token 和 Folder ID 供下次使用 (如果是第一次上傳)
            if not gofile_token:
                gofile_token = data.get('guestToken')
            if not gofile_folder_id:
                gofile_folder_id = data.get('parentFolder')

            print('\n✅ 上傳成功！')
            print('⬇️ 下載網址：', data['downloadPage'])
            if dev_mode:
                print(f"🔑 使用中 guestToken：{gofile_token}")
                print(f"📁 目標 parentFolder：{gofile_folder_id}")
                # print('詳細資料:', data)
        else:
            print(f"\n❌ 上傳失敗: {response_json}")

    except Exception as e:
        print(f"\n❌ 發生錯誤: {e}")

if filename:
    upload_huge_file(filename)
else:
    print("請輸入檔案名稱")

In [ ]:
#@title # **上傳到Gigafile**
import subprocess, shutil

# 只在未安裝時才安裝
if not shutil.which("gfile"):
    print("⏳ 正在安裝 gigafile...")
    subprocess.run(["pip", "install", "-q", "gigafile"], capture_output=True)
    print("✅ 安裝完成")

filename = "" #@param{type:"string"}

if filename:
    print(f"📁 檔案: {filename}")

    process = subprocess.Popen(
        ["gfile", "-n", "8", "upload", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,  # 合併 stderr 到 stdout
        bufsize=0
    )

    url = ""
    buffer = b""
    while True:
        byte = process.stdout.read(1)
        if not byte:
            break
        if byte in (b'\r', b'\n'):
            line = buffer.decode("utf-8", errors="replace").strip()
            buffer = b""
            if not line:
                continue
            if "http" in line:
                url = line
            else:
                # 用 \r 覆蓋同一行，:<80 補空白清除殘留字元
                print(f"\r⏳ {line:<80}", end="", flush=True)
        else:
            buffer += byte

    # 處理最後殘留的 buffer
    if buffer:
        line = buffer.decode("utf-8", errors="replace").strip()
        if "http" in line:
            url = line

    process.wait()
    print()  # 進度結束後換行

    if url:
        print(f"✅ 上傳完成！")
        print(f"🔗 {url}")
    elif process.returncode != 0:
        print("❌ 上傳失敗")
else:
    print("⚠️ 請輸入檔案名稱")

In [ ]:
#@title # **上傳到ViKiNG FiLE**
filename = "" #@param {type:"string"}

import os
import sys
import time
import requests
from tqdm.auto import tqdm

# ============================================================
#  工具函式
# ============================================================

def fmt_size(n):
    """把 bytes 轉成人類可讀的字串"""
    for u in ("B", "KB", "MB", "GB", "TB"):
        if abs(n) < 1024:
            return f"{n:.2f} {u}"
        n /= 1024
    return f"{n:.2f} PB"


def fmt_time(seconds):
    """秒數轉成 mm:ss 或 hh:mm:ss"""
    seconds = int(seconds)
    if seconds < 3600:
        return f"{seconds // 60:02d}:{seconds % 60:02d}"
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

# ============================================================
#  主流程
# ============================================================

def upload_to_viking(filepath: str):
    """
    使用 ViKiNG FiLE 的分片上傳 API：
      1. POST /api/get-upload-url  → 取得每個 part 的預簽名 URL
      2. PUT  每個 part（附進度條）
      3. POST /api/complete-upload → 完成上傳
    """

    API_BASE = "https://vikingfile.com/api"

    # ---------- 解析路徑 ----------
    filepath = os.path.expanduser(filepath)        # 支援 ~/...
    filepath = os.path.abspath(filepath)            # 相對路徑 → 絕對路徑

    if not os.path.isfile(filepath):
        print(f"❌ 找不到檔案：{filepath}")
        sys.exit(1)

    file_size = os.path.getsize(filepath)
    file_name = os.path.basename(filepath)

    print(f"📄 檔案名稱：{file_name}")
    print(f"📂 檔案路徑：{filepath}")
    print(f"📦 檔案大小：{fmt_size(file_size)}")
    print("─" * 50)

    # ---------- Step 1：取得上傳 URL ----------
    print("🔗 正在取得上傳 URL …")
    resp = requests.post(f"{API_BASE}/get-upload-url", data={"size": file_size})

    if resp.status_code != 200:
        print(f"❌ 取得上傳 URL 失敗 (HTTP {resp.status_code})")
        print(resp.text)
        sys.exit(1)

    info = resp.json()
    upload_id  = info["uploadId"]
    key        = info["key"]
    part_size  = info["partSize"]
    num_parts  = info["numberParts"]
    urls       = info["urls"]

    print(f"✅ 分片數量：{num_parts}，每片大小：{fmt_size(part_size)}")
    print("─" * 50)

    # ---------- Step 2：逐片上傳 ----------
    parts = []
    start_time = time.time()

    total_bar = tqdm(
        total=file_size,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        desc="📤 總上傳進度",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]",
        colour="green",
    )

    SEND_CHUNK = 1024 * 1024  # 每次送 1 MB

    class ProgressReader:
        """把 bytes 包裝成可迭代的 generator，同時更新進度條"""
        def __init__(self, data, chunk_size, bar):
            self.data = data
            self.chunk_size = chunk_size
            self.bar = bar
            self.offset = 0
            self.total = len(data)

        def __iter__(self):
            return self

        def __next__(self):
            if self.offset >= self.total:
                raise StopIteration
            end = min(self.offset + self.chunk_size, self.total)
            chunk = self.data[self.offset:end]
            self.offset = end
            self.bar.update(len(chunk))
            return chunk

        def __len__(self):
            return self.total

    with open(filepath, "rb") as f:
        for i in range(num_parts):
            part_number = i + 1
            this_part_size = part_size if part_number < num_parts else file_size - part_size * (num_parts - 1)

            print(f"\n📦 Part {part_number}/{num_parts}（{fmt_size(this_part_size)}）")
            chunk_data = f.read(this_part_size)

            reader = ProgressReader(chunk_data, SEND_CHUNK, total_bar)

            # 重試機制（最多 3 次）
            max_retries = 3
            for attempt in range(1, max_retries + 1):
                try:
                    put_resp = requests.put(
                        urls[i],
                        data=reader,
                        headers={"Content-Length": str(this_part_size)},
                        timeout=600,
                    )
                    put_resp.raise_for_status()
                    break
                except (requests.exceptions.RequestException, IOError) as e:
                    print(f"\n⚠️  Part {part_number} 上傳失敗（第 {attempt} 次）：{e}")
                    if attempt == max_retries:
                        total_bar.close()
                        print("❌ 已達最大重試次數，上傳中止。")
                        sys.exit(1)
                    reader = ProgressReader(chunk_data, SEND_CHUNK, total_bar)
                    total_bar.update(-this_part_size)
                    time.sleep(2 ** attempt)

            etag = put_resp.headers.get("ETag", "").strip('"')
            if not etag:
                total_bar.close()
                print(f"❌ Part {part_number} 沒有回傳 ETag！")
                print(f"   Response headers: {dict(put_resp.headers)}")
                sys.exit(1)

            parts.append({"PartNumber": part_number, "ETag": etag})
            print(f"   ✅ ETag: {etag[:16]}…")

    total_bar.close()
    elapsed = time.time() - start_time
    speed = file_size / elapsed if elapsed > 0 else 0
    print("─" * 50)
    print(f"⏱️  上傳耗時：{fmt_time(elapsed)}｜平均速度：{fmt_size(speed)}/s")
    print("─" * 50)

    # ---------- Step 3：完成上傳 ----------
    print("🔒 正在完成上傳 …")

    complete_data = {
        "key": key,
        "uploadId": upload_id,
        "name": file_name,
    }
    for idx, p in enumerate(parts):
        complete_data[f"parts[{idx}][PartNumber]"] = p["PartNumber"]
        complete_data[f"parts[{idx}][ETag]"] = p["ETag"]

    resp = requests.post(f"{API_BASE}/complete-upload", data=complete_data)

    if resp.status_code != 200:
        print(f"❌ 完成上傳失敗 (HTTP {resp.status_code})")
        print(resp.text)
        sys.exit(1)

    result = resp.json()

    # ---------- 結果 ----------
    print()
    print("═" * 50)
    print("🎉 上傳成功！")
    print("═" * 50)
    print(f"  📄 檔案名稱：{result.get('name', 'N/A')}")
    print(f"  📦 檔案大小：{fmt_size(int(result.get('size', 0)))}")
    print(f"  🔑 Hash    ：{result.get('hash', 'N/A')}")
    print(f"  🔗 URL     ：{result.get('url', 'N/A')}")
    print("═" * 50)

    return result


# ============================================================
#  執行
# ============================================================

if not filename.strip():
    print("⚠️  請在上方填入檔案名稱或路徑（filename）！")
else:
    upload_to_viking(filepath=filename.strip())

In [ ]:
#@title # **上傳到⁠Rootz**
import requests
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
#@markdown **Rootz上限 [25](https://rootz.so/docs) GB**

filename = "" #@param{type:"string"}
API_KEY = ''  # 從控制台設定中取得
BASE_URL = 'https://www.rootz.so'
MULTIPART_THRESHOLD = 4 * 1024 * 1024  # 4MB

def get_optimal_parallelism(file_size):
    """根據檔案大小決定最佳的平行上傳數量"""
    if file_size > 50 * 1024**3:  # > 50GB
        return 3
    elif file_size > 10 * 1024**3:  # > 10GB
        return 4
    elif file_size > 1 * 1024**3:  # > 1GB
        return 5
    else:
        return 6

def upload_small_file(file_path, folder_id=None):
    """直接上傳小型檔案（< 4MB）"""
    url = f'{BASE_URL}/api/files/upload'
    file_size = os.path.getsize(file_path)
    file_name = os.path.basename(file_path)

    print(f"📤 正在上傳 {file_name}（{file_size / (1024**2):.2f} MB）...")

    headers = {}
    if API_KEY:
        headers['Authorization'] = f'Bearer {API_KEY}'

    with open(file_path, 'rb') as f:
        files = {'file': (file_name, f)}
        data = {}
        if folder_id:
            data['folderId'] = folder_id

        response = requests.post(url, headers=headers, files=files, data=data)

    response.raise_for_status()
    result = response.json()
    if not result.get('success'):
        raise Exception(result.get('error', '上傳失敗'))

    print(f"✅ 上傳完成！")
    return result['data']

def upload_large_file_parallel(file_path, folder_id=None):
    """使用平行分段上傳大型檔案（>= 4MB）以達到最高速度"""
    file_size = os.path.getsize(file_path)
    file_name = os.path.basename(file_path)

    headers = {'Content-Type': 'application/json'}
    if API_KEY:
        headers['Authorization'] = f'Bearer {API_KEY}'

    # 1. 初始化分段上傳
    print(f"📦 正在初始化 {file_name} 的上傳（{file_size / (1024**3):.2f} GB）...")
    init_res = requests.post(
        f'{BASE_URL}/api/files/multipart/init',
        headers=headers,
        json={
            'fileName': file_name,
            'fileSize': file_size,
            'fileType': 'application/octet-stream',
            'folderId': folder_id
        }
    )
    init_res.raise_for_status()
    init_data = init_res.json()
    upload_id = init_data['uploadId']
    key = init_data['key']
    chunk_size = init_data['chunkSize']
    total_parts = init_data['totalParts']

    print(f"✅ 上傳已初始化：{total_parts} 個分段 × {chunk_size / (1024**2):.1f} MB")

    # 2. 預先取得所有預簽名 URL
    print(f"🔗 正在取得 {total_parts} 個分段的預簽名 URL...")
    urls_res = requests.post(
        f'{BASE_URL}/api/files/multipart/batch-urls',
        headers={'Content-Type': 'application/json'},
        json={'key': key, 'uploadId': upload_id, 'totalParts': total_parts}
    )
    urls_res.raise_for_status()
    urls_data = urls_res.json()
    if not urls_data.get('success'):
        raise Exception(urls_data.get('error', '取得上傳 URL 失敗'))

    urls_dict = urls_data['urls']
    print(f"✅ 已取得所有預簽名 URL")

    # 3. 使用平行方式上傳分段以達到最高速度
    parallelism = get_optimal_parallelism(file_size)
    print(f"\n🚀 正在以 {parallelism} 倍平行度上傳 {total_parts} 個分段...")
    start_time = time.time()
    uploaded_parts = []
    completed_parts = 0

    def upload_part(part_number):
        """上傳單一分段"""
        url = urls_dict[str(part_number)]

        with open(file_path, 'rb') as f:
            f.seek((part_number - 1) * chunk_size)
            chunk = f.read(chunk_size)

        # 帶有重試邏輯的上傳
        max_retries = 3
        for attempt in range(max_retries):
            try:
                res = requests.put(url, data=chunk, timeout=300)
                res.raise_for_status()
                etag = res.headers.get('ETag', '').strip('"')
                return {'partNumber': part_number, 'etag': etag}
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)  # 指數退避

    # 使用 ThreadPoolExecutor 進行平行上傳
    with ThreadPoolExecutor(max_workers=parallelism) as executor:
        futures = {executor.submit(upload_part, i): i for i in range(1, total_parts + 1)}

        for future in as_completed(futures):
            part_number = futures[future]
            try:
                result = future.result()
                uploaded_parts.append(result)
                completed_parts += 1

                # 更新進度
                elapsed = time.time() - start_time
                uploaded_bytes = completed_parts * chunk_size
                speed = uploaded_bytes / elapsed / (1024**2) if elapsed > 0 else 0
                progress = (completed_parts / total_parts) * 100
                eta = (total_parts - completed_parts) * chunk_size / (uploaded_bytes / elapsed) if uploaded_bytes > 0 else 0

                # 進度條
                bar_length = 30
                filled = int(bar_length * completed_parts / total_parts)
                bar = '█' * filled + '░' * (bar_length - filled)

                print(f"\r[{bar}] {progress:.1f}% | {completed_parts}/{total_parts} 個分段 | "
                      f"{speed:.1f} MB/s | 預估剩餘時間：{eta:.0f} 秒", end='', flush=True)
            except Exception as e:
                print(f"\n❌ 上傳分段 {part_number} 失敗：{e}")
                raise

    elapsed_total = time.time() - start_time
    avg_speed = file_size / elapsed_total / (1024**2)
    print(f"\n✅ 上傳完成，耗時 {elapsed_total:.1f} 秒（平均速度：{avg_speed:.1f} MB/s）")

    uploaded_parts.sort(key=lambda x: x['partNumber'])

    # 4. 完成分段上傳
    print(f"🔄 正在完成上傳...")
    complete_res = requests.post(
        f'{BASE_URL}/api/files/multipart/complete',
        headers=headers,
        json={
            'key': key,
            'uploadId': upload_id,
            'parts': uploaded_parts,
            'fileName': file_name,
            'fileSize': file_size,
            'contentType': 'application/octet-stream',
            'folderId': folder_id
        }
    )
    complete_res.raise_for_status()
    result = complete_res.json()
    if not result.get('success'):
        raise Exception(result.get('error'))

    print(f"✅ 檔案已儲存至資料庫")
    return result['file']

def upload_file(file_path, folder_id=None):
    """主要上傳函式 - 自動處理任何檔案大小並達到最佳效能"""
    file_size = os.path.getsize(file_path)

    if file_size < MULTIPART_THRESHOLD:
        return upload_small_file(file_path, folder_id)
    else:
        return upload_large_file_parallel(file_path, folder_id)


try:
  result = upload_file(filename)
  print(f"\n🎉 成功！分享連結：{BASE_URL}/d/{result['shortId']}")
except Exception as e:
  print(f'❌ 上傳失敗：{e}')

In [ ]:
#@title # **上傳到transfer.it**
import os
import subprocess
from tqdm.notebook import tqdm

#@markdown ### 檔案設定
#@markdown 請輸入檔案路徑，多個檔案請以**空白**分隔。<br>
#@markdown 若檔名包含空白，請使用雙引號包起來 (例如 `"my file.zip"`)。<br>
#@markdown 亦支援萬用字元 (例如 `*.jpg` 或 `/content/DL/*`)。
file_paths = "" #@param{type:"string"}

#@markdown ### 上傳模式
#@markdown 勾選此項將為每個檔案產生獨立的下載連結 (Separate Links)。
separate_links = False #@param {type:"boolean"}

# --- 安裝與環境設定 ---
repo_path = "/content/transfer-it"

def run_command(command, cwd=None, desc=""):
    """執行指令並更新進度條描述，若失敗則拋出錯誤"""
    try:
        # 使用 subprocess 執行指令，並隱藏一般輸出，只顯示錯誤
        subprocess.run(
            command,
            cwd=cwd,
            check=True,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
    except subprocess.CalledProcessError as e:
        print(f"\n❌ 錯誤發生於: {desc}")
        print(f"錯誤訊息: {e.stderr.decode('utf-8')}")
        raise e

if not os.path.exists(repo_path):
    # 定義安裝步驟：(描述, 指令, 執行路徑)
    steps = [
        ("正在下載儲存庫...", f"git clone https://github.com/chikenscrach/transfer-it.git", "/content"),
        ("正在安裝 uv 套件管理器...", "pip install uv", "/content"),
        ("正在安裝 Python 依賴...", "uv pip install -r requirements.txt", repo_path),
        ("正在安裝 Chromium 瀏覽器...", "playwright install chromium", repo_path),
        ("正在安裝系統依賴...", "playwright install-deps", repo_path),
    ]

    print("檢測到初次執行，開始安裝環境...")

    # 建立進度條
    with tqdm(total=len(steps), desc="安裝進度", unit="step") as pbar:
        for desc, cmd, work_dir in steps:
            pbar.set_description(desc) # 更新進度條文字
            run_command(cmd, cwd=work_dir, desc=desc)
            pbar.update(1) # 進度 +1

    print("\n✅ 環境安裝完成！")
else:
    # 確保工作目錄正確
    os.chdir(repo_path)

# --- 執行上傳指令 ---

# 判斷是否加上 --separate 參數
flags = "--separate" if separate_links else ""

# 確保在正確目錄執行
os.chdir(repo_path)

print(f"🚀 開始上傳: {file_paths} ...")
# 執行 Python 指令
!python transferit_zhtw.py {flags} {file_paths}

In [ ]:
#@title # 上傳到Google Drive
#@markdown **資料夾位置記得加 "/"**

google_drive_path = "ColabDownloads/" #@param {type:"string"}

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')

!rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}

print("\033[32mDone!\033[0m")

In [ ]:
#@title # **上傳到catbox**
#@markdown **catbox上限 [200](https://catbox.moe/) MB**

filename = "" #@param{type:"string"}

!curl -F "reqtype=fileupload" -F "fileToUpload=@{filename}" https://catbox.moe/user/api.php

---
# 其他

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")

In [ ]:
# 自行發揮

---
# 以下小工具


In [ ]:
#@title # Colab終端機
#@markdown [colab-xterm](https://github.com/InfuseAI/colab-xterm)
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
#@title # Telegram 工具
#@markdown [tdl](https://github.com/iyear/tdl) [文件](https://docs.iyear.me/tdl/)
!curl -sSL https://docs.iyear.me/tdl/install.sh | sudo bash
# upload tdl
# unzip tdl.zip -d ~/.tdl

In [ ]:
#@title # Mega帳號產生器
#@markdown 使用 [MEGAcmd](https://github.com/meganz/MEGAcmd) 和 [mail.tm](https://mail.tm/) API 產生帳號

import secrets, string, json, re, time, subprocess, os

# ========================
# 工具函數
# ========================

def run_cmd(cmd, capture=True, check=False):
    """執行 shell 命令並回傳 stdout"""
    result = subprocess.run(
        cmd, shell=True,
        capture_output=capture, text=True
    )
    if check and result.returncode != 0:
        raise RuntimeError(f"指令失敗: {cmd}\nstderr: {result.stderr}")
    return result.stdout.strip() if capture else None


def curl_json(method, url, headers=None, data=None):
    """發送 HTTP 請求並解析 JSON 回應"""
    cmd = f'curl -s -X {method} "{url}"'
    if headers:
        for k, v in headers.items():
            cmd += f' -H "{k}: {v}"'
    if data:
        cmd += f" -d '{json.dumps(data)}'"
    output = run_cmd(cmd)
    try:
        return json.loads(output)
    except json.JSONDecodeError:
        print(f"⚠️ JSON 解析失敗，原始回應:\n{output}")
        return None


def generate_account(domain, username_length=8, password_length=16):
    """產生隨機帳號（使用密碼學安全的隨機數）"""
    username = ''.join(secrets.choice(string.ascii_lowercase + string.digits)
                       for _ in range(username_length))
    # 確保密碼包含各類字元
    password_chars = string.ascii_letters + string.digits + "!@#$%^&*()"
    password = ''.join(secrets.choice(password_chars)
                       for _ in range(password_length))
    return {"address": f"{username}@{domain}", "password": password}


def wait_for_email(token, max_retries=15, interval=5):
    """等待郵件到達，帶重試機制"""
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    for attempt in range(1, max_retries + 1):
        print(f"  📧 等待郵件... ({attempt}/{max_retries})")
        resp = curl_json("GET", "https://api.mail.tm/messages", headers=headers)
        if resp and resp.get("hydra:member"):
            messages = resp["hydra:member"]
            if len(messages) > 0 and messages[0].get("downloadUrl"):
                return messages[0]["downloadUrl"]
        time.sleep(interval)
    return None


def extract_confirm_link(filepath):
    """從郵件檔案中提取 MEGA 確認連結"""
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    # 支援新舊兩種連結格式
    match = re.search(r'https://mega\.nz/(#confirm|confirm)\S+', content)
    return match.group() if match else None


# ========================
# 主流程
# ========================

%cd /content

# 1. 安裝 MEGAcmd
print("=" * 50)
print("📦 安裝 MEGAcmd...")
print("=" * 50)
!apt-get install -y jq > /dev/null 2>&1
!wget -q https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb \
    && sudo apt install -y "$PWD/megacmd-xUbuntu_22.04_amd64.deb" > /dev/null 2>&1

# 初始化 MEGAcmd server（防止後續指令卡住）
run_cmd("mega-ls", check=False)
time.sleep(2)

# 2. 取得 mail.tm 可用域名
print("\n📬 取得 mail.tm 域名...")
domain_resp = curl_json("GET", "https://api.mail.tm/domains")
if not domain_resp or not domain_resp.get("hydra:member"):
    raise RuntimeError("❌ 無法取得 mail.tm 域名")
domain = domain_resp["hydra:member"][0]["domain"]
print(f"  ✅ 域名: {domain}")

# 3. 產生帳號
account_info = generate_account(domain)
print(f"  ✅ 帳號: {account_info['address']}")

# 4. 在 mail.tm 建立信箱
print("\n📮 建立臨時信箱...")
create_resp = curl_json("POST", "https://api.mail.tm/accounts",
                        headers={"Content-Type": "application/json"},
                        data=account_info)
if not create_resp or "id" not in create_resp:
    raise RuntimeError(f"❌ 建立信箱失敗: {create_resp}")
print(f"  ✅ 信箱建立成功")

# 5. 取得 mail.tm Token
token_resp = curl_json("POST", "https://api.mail.tm/token",
                       headers={"Content-Type": "application/json"},
                       data=account_info)
if not token_resp or "token" not in token_resp:
    raise RuntimeError(f"❌ 取得 Token 失敗: {token_resp}")
token = token_resp["token"]

# 6. MEGA 註冊
print("\n🔐 MEGA 註冊中...")
signup_output = run_cmd(
    f'mega-signup "{account_info["address"]}" "{account_info["password"]}"'
)
print(f"  {signup_output}")

# 7. 等待確認郵件
print("\n⏳ 等待確認郵件...")
download_url = wait_for_email(token)
if not download_url:
    raise RuntimeError("❌ 超時：未收到確認郵件")

# 8. 下載並解析郵件
mail_path = "/content/mail.txt"
run_cmd(
    f'curl -s -H "Authorization: Bearer {token}" '
    f'-X GET "https://api.mail.tm{download_url}" -o {mail_path}'
)

confirm_url = extract_confirm_link(mail_path)
if not confirm_url:
    raise RuntimeError("❌ 找不到確認連結，郵件內容可能有變")
print(f"  ✅ 確認連結已找到")

# 9. 確認並登入
print("\n✅ 確認帳號...")
run_cmd(f'mega-confirm "{confirm_url}" "{account_info["address"]}" "{account_info["password"]}"')

print("🔑 登入中...")
run_cmd(f'mega-login "{account_info["address"]}" "{account_info["password"]}"')

# 10. 清理
os.remove(mail_path)
deb_files = [f for f in os.listdir("/content") if f.endswith(".deb")]
for f in deb_files:
    os.remove(os.path.join("/content", f))

# 11. 結果
print("\n" + "=" * 50)
print("🎉 帳號建立成功！")
print("=" * 50)
print(f"  📧 帳號： {account_info['address']}")
print(f"  🔑 密碼： {account_info['password']}")
print("=" * 50)
print("\n常用指令：")
print("  上傳：!mega-put 檔案")
print("  下載：!mega-get link")
print("  分享：!echo Yes | mega-export -a 檔案")

In [ ]:
#@title # Gdrive
#@markdown [gdrive](https://github.com/glotlabs/gdrive)
%cd /content
!wget https://github.com/glotlabs/gdrive/releases/download/3.9.1/gdrive_linux-x64.tar.gz
!tar zxvf gdrive_linux-x64.tar.gz

# 如何登入自己的帳號？請看以下網址
# https://github.com/glotlabs/gdrive/blob/main/docs/create_google_api_credentials.md

# !./gdrive account import account.tar
# !rm gdrive_linux-x64.tar.gz account.tar
%cd /content/DL
# 下載
# !/content/gdrive files download "id" --recursive
# https://drive.google.com/file/d/"這裡面的就是ID"/view
# 上傳
# !/content/gdrive files upload [OPTIONS] [FILE_PATH]

In [ ]:
#@title # YouTube影片下載
#@markdown [po_token](https://github.com/Brainicism/bgutil-ytdlp-pot-provider)

import os
yt_url = "https://youtu.be/dQw4w9WgXcQ" #@param {type:"string"}

if not os.path.exists("/root/bgutil-ytdlp-pot-provider"):
  !curl -fsSL https://deno.land/install.sh | sh
  !ln -s /root/.deno/bin/deno /usr/local/bin/deno
  !pip install uv
  !uv pip install -U yt-dlp bgutil-ytdlp-pot-provider yt-dlp-ejs
  %cd ~
  !git clone --single-branch --branch 1.2.2 https://github.com/Brainicism/bgutil-ytdlp-pot-provider.git
  %cd bgutil-ytdlp-pot-provider/server/
  !npm install
  !npx tsc
  %cd /content

#@markdown ※若遇`HTTP Error 403: Forbidden`或其他驗證Error可嘗試`中斷連線並刪除執行階段`後重新執行
!yt-dlp --ignore-errors --output "%(title)s.%(ext)s" --verbose --merge-output-format mp4 --embed-thumbnail --embed-metadata --embed-info-json --extractor-args "youtubepot-bgutilscript:script_path=/root/bgutil-ytdlp-pot-provider/server/build/generate_once.js" "{yt_url}"

In [ ]:
#@title # YouTube直播下載
#@markdown [livestream_dl](https://github.com/CanOfSocks/livestream_dl)
import os

if not os.path.exists("/content/livestream_dl"):
  %cd /content
  !git clone https://github.com/CanOfSocks/livestream_dl.git
  %cd livestream_dl
  !pip install uv
  !curl -fsSL https://deno.land/install.sh | sh
  !ln -s /root/.deno/bin/deno /usr/local/bin/deno
  !uv pip install -r requirements.txt yt-dlp chat-downloader yt-dlp-ejs

yt_url = "" #@param{type:"string"}

%cd /content/livestream_dl
!python runner.py --threads 4 --dash --m3u8 --write-thumbnail --embed-thumbnail --wait-for-video "60:600" --clean-info-json --remove-ip-from-json --live-chat --resolution "best" --write-info-json --log-level "INFO" -- "{yt_url}"

In [ ]:
#@title # Speedtest
#@markdown [SPEEDTEST](https://www.speedtest.net/zh-Hant/apps/cli)
!curl -s https://packagecloud.io/install/repositories/ookla/speedtest-cli/script.deb.sh | sudo bash
!apt-get install speedtest
!echo YES | speedtest